In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# open file
from IPython.display import Image
import os
import PIL

# model CNN (Deep learning network)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense,\
GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Create function plot loss function and accuracy score graph
def plot_graph(model_values):
    ''' 
    Input : Model_values of keras.callbacks.History
    Return : Graph of Loss function and accuracy score between training dataset and vaildation dataset
    '''
    # Subplots
    fig, ax = plt.subplots(1, 2, figsize=(14,5))
    
    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(model_values.history['loss'], label='Training Loss');
    plt.plot(model_values.history['val_loss'], label='Testing Loss');
    plt.legend(fontsize=12, loc='upper right')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss');
    
    # Plot MSE
    plt.subplot(1, 2, 2)
    
    plt.plot(model_values.history['accuracy'], label='Training Accuracy')
    plt.plot(model_values.history['val_accuracy'], label='Validation Accuracy')
    
    plt.legend(fontsize=12, loc='lower right')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy');

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6985485098360419511
xla_global_id: -1
]


In [4]:
gpus = tf.config.list_physical_devices("GPU")
if gpus:
  print("Found a GPU with the name:", gpu)
else:
  print("Failed to detect a GPU.")

Failed to detect a GPU.


In [5]:
# Open path file of dataset 
# We prepare actucl diretories form to training in 01.Preprocessing datasets notebook
dataset_path_new = "dataset_train_valid_test"

train_dir = os.path.join(dataset_path_new, "train")
valid_dir = os.path.join(dataset_path_new, "valid")
test_dir = os.path.join(dataset_path_new, "test")

In [6]:
# Set parameter of image 
# We use input small size image because reduce size of model 
batch_size = 50 # Set the batch size for epoch cycle
img_height = 128 # Set the height of the picture
img_width = 128 # Set the width of the picture

# Rescale pixel to reduce image size before using in model
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)
data_gen_test = ImageDataGenerator(rescale=1/255.)

In [7]:
# Create training dataset 
train_dataset = data_gen_train.flow_from_directory(train_dir,
                                                   class_mode="categorical",
                                                   target_size=(img_height, img_width),
                                                   batch_size=batch_size)
 

# Create validation dataset 
valid_dataset = data_gen_valid.flow_from_directory(valid_dir,
                                                   class_mode="categorical",
                                                   target_size=(img_height, img_width),
                                                   batch_size=batch_size)

# Create testing dataset 
test_dataset = data_gen_test.flow_from_directory(test_dir,
                                                   class_mode="categorical",
                                                   target_size=(img_height, img_width),
                                                   batch_size=batch_size)

Found 6367 images belonging to 4 classes.
Found 686 images belonging to 4 classes.
Found 668 images belonging to 4 classes.


In [8]:
# Classes in training dataset
print(pd.Series(train_dataset.classes).value_counts(normalize = True).mul(100))
print(train_dataset.class_indices)

3    28.647715
0    26.825821
1    25.490812
2    19.035653
Name: proportion, dtype: float64
{'cocci': 0, 'healthy': 1, 'ncd': 2, 'salmo': 3}


In [9]:
# Classes in validation dataset
pd.Series(valid_dataset.classes).value_counts()
# Class 0 : cocci
# Class 1 : healthy
# Class 2 : ncd 
# Class 3 : salmo

1    240
3    207
0    199
2     40
Name: count, dtype: int64

In [10]:
# Classes in training dataset
pd.Series(test_dataset.classes).value_counts()

3    245
0    196
1    194
2     33
Name: count, dtype: int64

In [11]:
images, labels_class = next(train_dataset)
classes = train_dataset.class_indices
print(images[0].max())
print(images[0].min())

0.92549026
0.039215688


In [12]:
# Training model
# Compile the model
model = Sequential()

# Input layer
model.add(Conv2D(16,(3,3), activation='relu',
                 kernel_initializer='he_uniform', 
                 padding='same', input_shape=(128,128,3))) # filter image by dot product in matrix to find the object in picture
# Hidden layers
model.add(MaxPooling2D(pool_size=(2,2))) # reduce stucture of image even if object slightly move but the result is same  
model.add(Conv2D(32,(3,3), activation='relu',
                 kernel_initializer='he_uniform', 
                 padding='same')) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten()) # convert all the resultant 2-Dimensional arrays into a single long continuous linear vector
model.add(Dense(64, activation='relu',
                kernel_initializer='he_uniform'))

# avoid overfitting!
model.add(Dropout(0.5))

# Output layer
model.add(Dense(4, activation='softmax'))

# Compile the model         
model.compile(loss="categorical_crossentropy", 
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [13]:
# we have 2,102,564 nodes for training
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 64)                2

In [14]:
# Save checkpoints during training
checkpoint_path = "../model/cnn/cnn_model_cp/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [15]:
# training model 
# make sure you truely save checkpoint_path
history = model.fit(train_dataset,
                              epochs=25,
                              validation_data=valid_dataset,
                              callbacks=[cp_callback])

Epoch 1/25
128/128 [==============================] - ETA: 0s - loss: 1.8384 - accuracy: 0.3471
Epoch 1: saving model to ../model/cnn/cnn_model_cp\cp.ckpt
128/128 [==============================] - 430s 3s/step - loss: 1.8384 - accuracy: 0.3471 - val_loss: 0.9772 - val_accuracy: 0.5262
Epoch 2/25
128/128 [==============================] - ETA: 0s - loss: 1.1140 - accuracy: 0.4450
Epoch 2: saving model to ../model/cnn/cnn_model_cp\cp.ckpt
128/128 [==============================] - 415s 3s/step - loss: 1.1140 - accuracy: 0.4450 - val_loss: 0.8745 - val_accuracy: 0.5481
Epoch 3/25
128/128 [==============================] - ETA: 0s - loss: 1.0632 - accuracy: 0.4475
Epoch 3: saving model to ../model/cnn/cnn_model_cp\cp.ckpt
128/128 [==============================] - 418s 3s/step - loss: 1.0632 - accuracy: 0.4475 - val_loss: 0.7968 - val_accuracy: 0.5816
Epoch 4/25
128/128 [==============================] - ETA: 0s - loss: 0.9950 - accuracy: 0.5048
Epoch 4: saving model to ../model/cnn/cnn_m

In [18]:
model.save("../model/cnn/cnn_model.h5")

In [19]:
hist_df = pd.DataFrame(history.history) 
hist_df.head()

,loss,accuracy,val_loss,val_accuracy
0,1.838431,0.347102,0.977244,0.526239
1,1.113976,0.444951,0.874544,0.548105
2,1.063180,0.447463,0.796834,0.581633
3,0.994971,0.504790,0.746442,0.779883
4,0.937736,0.574839,0.697862,0.836735


In [20]:
# save history our model to csv: 
hist_csv_file = '../model/cnn/history_cnn16.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)